In [ ]:
# -*- coding: utf-8 -*-
#leemos las librerias para poder aplicar distintas funciones
library(readxl) 
library(dplyr)
library(stringr)
library(writexl)

In [ ]:
#guardamos los archivos
Victoria <- read.csv("VICTORIA.csv")

In [ ]:
names(Victoria) #muestra las variables
#cambiamos los nombres de las variables para un mejor entendimiento
Victoria <- rename(Victoria, c("CodigoUnicoEscolar" = "CUE", "ModEnseñanza" = "ModEnsenaza", "año" = "anioLectivo", "esMultiaño" ="esMultianio", "IdModEnseñanza" = "IdModEnsenanza","NivelEnseñanza" = "NivelEnsenanza"))
str(Victoria) #muestra los nombres del dataset, nos ayuda a identificar el tipo de variables

In [ ]:
#descubrimos que al intentar modificar el nombre de una varible, esta se encuentra dos veces en el conjunto de datos
Victoria$ddivision <- NULL #entonces procedemos a eliminar una columna
names(Victoria) #verificamos si se elimino correctamente

In [ ]:
#reemplazamos las notas "EPAC", "UP" y "PA" 
Victoria$nota[Victoria$nota %in% c("EPAC")] <- 5
Victoria$nota[Victoria$nota %in% c("PA")] <- 6
Victoria$nota[Victoria$nota %in% c("UP")] <- 7
Victoria$nota <- suppressWarnings(as.numeric(Victoria$nota)) #convertimos la variable nota a tipo numérico

In [ ]:
#ahora podemos dividir el dataset en dos
Victoria_primaria <- filter(Victoria, NivelEnseñanza == "PRIMARIO" & ( ModEnseñanza == "Comun"| ModEnseñanza == "Escuelas NINA") & Modalidad == "Primaria" & esMultiaño == "N") #se guardan los alumnos que cursan la primaria
Victoria_secundaria <- filter(Victoria, NivelEnseñanza == "SECUNDARIO" & (ModEnseñanza == "Comun" | ModEnseñanza == "Técnica" | ModEnseñanza =="Agrotécnica") & esMultiaño == "N" & Modalidad == "Secundaria") #se guardan los alumnos que cursan la secundaria
length(unique(Victoria_primaria$CodigoUnicoEscolar)) #cantidad de escuelas primarias en Victoria
length(unique(primaria_2023$CUE)) #total de escuelas primarias
length(unique(Victoria_secundaria$CodigoUnicoEscolar))#cantidad de escuelas secundarias en Victoria
length(unique(secundaria_2023$CUE))#total de escuelas secundarias

In [ ]:
dim(Victoria_primaria)
dim(Victoria_secundaria)

In [ ]:
PrimerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_primaria <- rename(PrimerT_primaria,NotasPrimerT = nota)
SegundoT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_primaria <- rename(SegundoT_primaria,NotasSegundoT = nota)
TercerT_primaria <- filter(Victoria_primaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_primaria <- rename(TercerT_primaria,NotasTercerT = nota)

In [ ]:
aux <- merge(x = PrimerT_primaria, y = SegundoT_primaria, by = c("Documento", "asignatura"))
auxiliar <- merge(x = aux, y = TercerT_primaria, by = c("Documento", "asignatura"))

In [ ]:
Notas_Victoria_primaria <- select(auxiliar, 'CodigoUnicoEscolar', 'departamento', 'localidad', 'EsPrivada', 'turno' , 'curso' , 'periodoEvaluatorio' , 'Documento', 'asignatura','idalumno', 'observaciones', 'idSubOrganizacion', 'idDivision', 'NivelEnseñanza', 'orden','Nivel', 'esMultiaño', 'IdModEnseñanza', 'Modalidad', 'ModEnseñanza', 'año','NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [ ]:
Notas_Victoria_primaria$Promedio <-round ((as.numeric(Notas_Victoria_primaria$NotasPrimerT)+as.numeric(Notas_Victoria_primaria$NotasSegundoT)+as.numeric(Notas_Victoria_primaria$NotasTercerT))/3,2)
head(Notas_Victoria_primaria)

In [ ]:
xd <- filter(Notas_Victoria_primaria, NotasTercerT < 6)
xd

In [ ]:
Notas_Victoria_primaria <- Notas_Victoria_primaria %>% 
  mutate(Condición = case_when(
    Promedio >= 6 & NotasTercerT >= 6 ~ "Aprobado", 
    Promedio < 6 | NotasTercerT < 6 ~ "Rinde",      
    TRUE ~ "sin registro completo"                    
  ))

head(Notas_Victoria_primaria)

In [ ]:
table(Notas_Victoria_primaria$asignatura)
head(Notas_Victoria_primaria$asignatura,10)
Notas_Victoria_primaria$asignatura <- toupper(Notas_Victoria_primaria$asignatura)
head(Notas_Victoria_primaria$asignatura,10)

In [ ]:
Notas_Victoria_primaria$asignatura <- gsub("MATEMÁTICA|ACOMPAÑAMIENTO AL ESTUDIO MATEMÁTICA", "MATEMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("LITERATURA|ACOMPAÑAMIENTO AL ESTUDIO LENGUA", "LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO SOCIALES", "CIENCIAS SOCIALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("ACOMPAÑAMIENTO AL ESTUDIO NATURALES|TALLER VIDA EN LA NATURALEZA", "CIENCIAS NATURALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE ARTES VISUALES|TALLER DE LENGUAJE DE LAS ARTES VISUALES", "ARTES VISUALES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE DEPORTE|DEPORTE", "EDUCACION FISICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("E.D.I.", "EDI", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE MUSICA|TALLER DE FOLKLORE", "EDUCACION MUSICAL", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TIC|TALLER DE TIC Y RECREACIÓN DE LA IMAGEN|TALLER TIC|TICS", "TIC", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("IDIOMA EXTRANJERO|INGLÉS|TALLER DE INGLES|TALLER DE INGLÉS|TALLER DE LENGUA EXTRANJERA|INGLEs|TALLER INGLES|TALLER LENGUA EXTRANJERA (INGLES)", "INGLES", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER DE TEATRO|TALLER TEATRO", "TALLER  DE TEATRO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("INFORMÁTICA", "INFORMATICA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("TALLER LABORATORIO", "TALLER DE LABORATORIO", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN LENGUA Y CS. SOCIALES|T/A ESTUDIO (LENGUA-CS SOCIALES)|TALLER ACOMPAÑAMIENTO LENGUA Y CIENCIAS SOCIALES", "TALLER DE CS. SOCIALES Y LENGUA", Notas_Victoria_primaria$asignatura)
Notas_Victoria_primaria$asignatura <- gsub("AC.EN MATEMÁTICA Y CS. NATURALES|T/A ESTUDIO (MATEMATICA-CS NATURALES)|TALLER ACOMPAÑAMIENTO  MATEMÁTICA Y CS. NATURALES", "TALLER DE CS. NATURALES Y MATEMATICA", Notas_Victoria_primaria$asignatura)

In [ ]:
table(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$curso != "1-6 - A Prim", ]
table(Notas_Victoria_primaria$curso)

In [ ]:
Notas_Victoria_primaria <- Notas_Victoria_primaria[Notas_Victoria_primaria$Condición != "sin registro completo", ]

In [ ]:
Notas_Victoria_primaria$curso <- as.character(Notas_Victoria_primaria$curso)
Notas_Victoria_primaria$Año_curso <- str_sub(Notas_Victoria_primaria$curso,1, 1)
print(head(Notas_Victoria_primaria$Año_curso),2)
table(Notas_Victoria_primaria$Año_curso)

In [ ]:
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_primaria$Año_curso <- str_replace(Notas_Victoria_primaria$Año_curso, "6" , "SEXTO")

In [ ]:
Notas_Victoria_1Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "PRIMERO"))
math <- Notas_Victoria_1Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_1Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_2Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEGUNDO"))
math <- Notas_Victoria_2Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla2 <- Notas_Victoria_2Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla2

In [ ]:
Notas_Victoria_3Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "TERCERO"))
math <- Notas_Victoria_3Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla3 <- Notas_Victoria_3Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla3

In [ ]:
Notas_Victoria_4Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "CUARTO"))
math <- Notas_Victoria_4Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla4 <- Notas_Victoria_4Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla4

In [ ]:
Notas_Victoria_5Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "QUINTO"))
math <- Notas_Victoria_5Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla5 <- Notas_Victoria_5Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla5

In [ ]:
Notas_Victoria_6Grado <- distinct(filter(Notas_Victoria_primaria, Año_curso == "SEXTO"))
math <- Notas_Victoria_6Grado %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla6 <- Notas_Victoria_6Grado %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla6

In [ ]:
dim(Notas_Victoria_primaria)

In [ ]:
write_xlsx(Notas_Victoria_primaria, "C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/Trabajo Practico/VICTORIA_BASE_PRIMARIA_LIMPIA.xlsx")

In [ ]:
PrimerT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Primer Trimestre")
PrimerT_secundaria <- rename(PrimerT_secundaria,NotasPrimerT = nota)
SegundoT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Segundo Trimestre")
SegundoT_secundaria <- rename(SegundoT_secundaria,NotasSegundoT = nota)
TercerT_secundaria <- filter(Victoria_secundaria, periodoEvaluatorio == "Tercer Trimestre")
TercerT_secundaria <- rename(TercerT_secundaria,NotasTercerT = nota)

In [ ]:
aux2 <- merge(x = PrimerT_secundaria, y = SegundoT_secundaria, by = c("Documento", "asignatura"))
auxiliar2 <- merge(x = aux2, y = TercerT_secundaria, by = c("Documento", "asignatura"))

In [ ]:
Notas_Victoria_secundaria <- select(auxiliar2, 'CodigoUnicoEscolar', 'departamento', 'localidad', 'EsPrivada', 'turno' , 'curso' , 'periodoEvaluatorio' , 'Documento', 'asignatura','idalumno', 'observaciones', 'idSubOrganizacion', 'idDivision', 'NivelEnseñanza', 'orden','Nivel', 'esMultiaño', 'IdModEnseñanza', 'Modalidad', 'ModEnseñanza', 'año','NotasPrimerT','NotasSegundoT', 'NotasTercerT')

In [ ]:
Notas_Victoria_secundaria$Promedio <-round ((as.numeric(Notas_Victoria_secundaria$NotasPrimerT)+as.numeric(Notas_Victoria_secundaria$NotasSegundoT)+as.numeric(Notas_Victoria_secundaria$NotasTercerT))/3,2)
head(Notas_Victoria_secundaria)

In [ ]:
Notas_Victoria_secundaria <- Notas_Victoria_secundaria %>% mutate(Condición = case_when(
  Promedio >= 6 ~ "Aprobado",
  Promedio <=6  ~ "Rinde",
   TRUE~ "sin registro completo"))
head(Notas_Victoria_secundaria)

In [ ]:
table(Notas_Victoria_secundaria$asignatura)
head(Notas_Victoria_secundaria$asignatura,10)
Notas_Victoria_secundaria$asignatura <- toupper(Notas_Victoria_secundaria$asignatura)
head(Notas_Victoria_secundaria$asignatura,10)

In [ ]:
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*BIOLOGIA.*", "BIOLOGIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*MATEMATICA*", "MATEMATICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*LENGUA*", "LENGUA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*EXTRANJERA*", "INGLES")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*GEOGRAFIA.*", "GEOGRAFIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*HISTORIA*", "HISTORIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*BIOLOGIA.*", "BIOLOGIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ETICA*", "ETICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*MUSICA*", "MUSICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ETICA*", "ETICA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*FILOSOFIA*", "FILOSOFIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*ECONOMIA*", "ECONOMIA")
Notas_Victoria_secundaria$asignatura <- str_replace_all(Notas_Victoria_secundaria$asignatura, ".*FISICO QUIMICA*", "FISICA Y QUIMICA")
Notas_Victoria_secundaria$asignatura <- gsub("AUTO C.A.D", "C.A.D", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("DERECHO COMERCIAL", "DERECHO", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CEREALES Y OLEAGINOSAS|MARCO JURIDICO DE LOS CIENCIAS AGROPECUARIAS |PRODUCCION HORTICOLA|PRODUCCION DE PLANTAS EN VIVERO |PRODUCCION DE FORRAJES|PRODUCCION DE CERDOS|PRODUCCION DE CAPRINOS|PRODUCCION BOVINO PARA CARNE|PROCESOS PRODUCTIVOS|INSTALACIONES AGROPECUARIAS|CIENCIAS NATURALES APLICADAS A PRO. AGROPECUARIOS|INDUSTRIA DE PROD. LACTEOS DE BOVINO O CAPRINO|INDUSTRIA CARNE PORCINA", "CIENCIAS AGROPECUARIAS", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CIENCIAS NATURALES (FISICO QUIMICA)|FISICO Y QUIMICA)|CIENCIAS NATURALES FISICO-QUIMICA|FISICA Y QUIMICA|ESPACIO DE LA ORIENTACION EN CIENCIAS NATURALES (FISICO QUIMICA)", "FISICO Y QUIMICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("CIRCUITOS TURISTICOS I Y II|MARKETING TURISTICO|INTRODUCCION AL TURISMO|EMPRESAS TURISTICAS|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL II|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL I|INFRAESTRUCTURA TURISTICA Y TURISMO TERMAL|CIRCUITOS TURISTICOS II|DESARROLLO DE LOCALIDADES TURISTICAS", "TURISMO", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("ECONOMIA II", "ECONOMIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("ETICA Y CIUDADANA|ETICAS CONTEMPORANEAS I", "ETICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FILOSOFIA II", "FILOSOFIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FISICA APLICADA|TRABAJOS PRACTICOS DE FISICA APLICADA", "FISICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("FRANCÉS", "FRANCES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("INGLES (INGLES)|INGLES (INGLES)|INGLESS", "INGLES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("PORTUGUÉS", "PORTUGUES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("QUIMICA ANALITICA CUALITATIVA|QUIMICA APLICADA|TRABAJOS PRACTICOS DE QUIMICA ORGANICA II|TRABAJOS PRACTICOS DE QUIMICA ORGANICA |TRABAJOS PRACTICOS DE QUIMICA INORGANICA |TRABAJOS PRACTICOS DE QUIMICA INDUSTRIAL APLICADA |TRABAJOS PRACTICOS DE QUIMICA INDUSTRIAL|TRABAJOS PRACTICOS DE QUIMICA GENERAL|TRABAJOS PRACTICOS DE QUIMICA ANALITICA CUANTIT.|TRABAJOS PRACTICOS DE PROC. Y OPERACIONES QUIMICAS|TRABAJOS PRACTICOS DE PROC. Y OPERACIONES QUIMICAS|QUIMICA ANALITICA CUANTITATIVA|QUIMICA GENERAL|QUIMICA INORGANICA|QUIMICA ORGANICA II|TALLER MODULAR QUIMICA", "QUIMICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("TECNOLOGIA I|TECNOLOGIA II|TECNOLOGIA III|EDUCACION TECNOLOGICA", "TECNOLOGIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("TUTORIAS DISCIPLINARES", "TUTORIAS", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("GESTION DE LA PRODUCCION ARTISTICA", "ARTES VISUALES", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("HISTORIA)", "HISTORIA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("MATEMATICA|MATEMATICAO", "MATEMATICA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("LENGUA Y LITERATURA|LITERATURA ARGENTINA|LITERATURA LATINOAMERICANA", "LENGUA Y LITERATURA", Notas_Victoria_secundaria$asignatura)
Notas_Victoria_secundaria$asignatura <- gsub("INGLES\\s*\\(INGLES\\)", "INGLES", Notas_Victoria_secundaria$asignatura)

In [ ]:
table(Notas_Victoria_secundaria$asignatura)

In [ ]:
table(Notas_Victoria_secundaria$curso)
Notas_Victoria_secundaria <- Notas_Victoria_secundaria[Notas_Victoria_secundaria$curso != "CN - RURAL CNR", ]
table(Notas_Victoria_secundaria$curso)

In [ ]:
Notas_Victoria_secundaria <- Notas_Victoria_secundaria[Notas_Victoria_secundaria$Condición != "sin registro completo", ]

In [ ]:
Notas_Victoria_secundaria$curso <- as.character(Notas_Victoria_secundaria$curso)
Notas_Victoria_secundaria$Año_curso <- str_sub(Notas_Victoria_secundaria$curso,1, 1)
print(head(Notas_Victoria_secundaria$Año_curso),2)
table(Notas_Victoria_secundaria$Año_curso)

In [ ]:
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "1" , "PRIMERO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "2" , "SEGUNDO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "3" , "TERCERO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "4" , "CUARTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "5" , "QUINTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "6" , "SEXTO")
Notas_Victoria_secundaria$Año_curso <- str_replace(Notas_Victoria_secundaria$Año_curso, "7" , "SEPTIMO")

In [ ]:
Notas_Victoria_1Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "PRIMERO"))
math <- Notas_Victoria_1Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_1Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_2Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEGUNDO"))
math <- Notas_Victoria_2Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_2Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_3Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "TERCERO"))
math <- Notas_Victoria_3Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_3Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_4Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "CUARTO"))
math <- Notas_Victoria_4Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_4Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_5Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "QUINTO"))
math <- Notas_Victoria_5Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_5Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_6Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEXTO"))
math <- Notas_Victoria_6Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_6Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
Notas_Victoria_7Año <- distinct(filter(Notas_Victoria_secundaria, Año_curso == "SEPTIMO"))
math <- Notas_Victoria_7Año %>% group_by(CodigoUnicoEscolar,asignatura) %>%  filter(asignatura=="MATEMATICA")
math
tabla1 <- Notas_Victoria_7Año %>% group_by(asignatura) %>%  summarise(conteo=n())
tabla1

In [ ]:
dim(Notas_Victoria_secundaria)

In [ ]:
write_xlsx(Notas_Victoria_secundaria, "C:/Users/ayrto/OneDrive/Escritorio/Exploración de Datos Multivariados/Trabajo Practico/VICTORIA_BASE_SECUNDARIA_LIMPIA.xlsx")